In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}


for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample
        categories = [symbol_to_category.get(sym, 'Other') for sym in symbols]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

        print(f"Files created for: {patient_number}: {ecg_filename}, {annotations_filename}")
    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("All done!")

Files created for: 100: data_creation/100_ECG.csv, data_creation/100_Annotations.csv
Files created for: 101: data_creation/101_ECG.csv, data_creation/101_Annotations.csv
Files created for: 102: data_creation/102_ECG.csv, data_creation/102_Annotations.csv
Files created for: 103: data_creation/103_ECG.csv, data_creation/103_Annotations.csv
Files created for: 104: data_creation/104_ECG.csv, data_creation/104_Annotations.csv
Files created for: 105: data_creation/105_ECG.csv, data_creation/105_Annotations.csv
Files created for: 106: data_creation/106_ECG.csv, data_creation/106_Annotations.csv
Files created for: 107: data_creation/107_ECG.csv, data_creation/107_Annotations.csv
Files created for: 108: data_creation/108_ECG.csv, data_creation/108_Annotations.csv
Files created for: 109: data_creation/109_ECG.csv, data_creation/109_Annotations.csv
Files created for: 111: data_creation/111_ECG.csv, data_creation/111_Annotations.csv
Files created for: 112: data_creation/112_ECG.csv, data_creation/

In [3]:
def process_patient_data(patient_number, data_creation_dir="data_creation"):

    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]

    sampling_rate = 360  # Hz
    window_size_seconds = 3  # Seconds before and after annotation
    window_size_samples = window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        start_point = max(0, annotation_point - window_size_samples)
        end_point = min(len(ecg_df), annotation_point + window_size_samples)
        
        window_data = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        if len(window_data) < window_size_samples * 2:
            window_data = np.pad(window_data, (0, window_size_samples * 2 - len(window_data)), 'constant')
        
        patient_X.append(window_data)
        patient_Y.append(category)
    
    return patient_X, patient_Y

# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       [-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       [-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       ...,
       [-0.29 , -0.3  , -0.295, ...,  0.   ,  0.   ,  0.   ],
       [-0.29 , -0.29 , -0.28 , ...,  0.   ,  0.   ,  0.   ],
       [-0.215, -0.22 , -0.225, ...,  0.   ,  0.   ,  0.   ]])

In [5]:
Y

array(['Other', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U5')